# Interpretabilidade de modelos

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Interpretabilidade de modelos
- 2) Modelos naturalmente interpretáveis
- 3) LIME
- 4) SHAP

Para a parte prática da aula, precisaremos de algumas bibliotecas adicionais:

`!pip install lime`

`!pip install shap`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

def metricas_classificacao(estimator, X_train, X_test, y_train, y_test):
    
    # ============================================

    print("\nMétricas de avaliação de treino:")

    y_pred_train = estimator.predict(X_train)

    print(confusion_matrix(y_train, y_pred_train))

    ConfusionMatrixDisplay.from_predictions(y_train, y_pred_train)
    plt.show()

    print(classification_report(y_train, y_pred_train))

    # ============================================

    print("\nMétricas de avaliação de teste:")

    y_pred_test = estimator.predict(X_test)

    print(confusion_matrix(y_test, y_pred_test))

    ConfusionMatrixDisplay.from_predictions(y_test, y_pred_test)
    plt.show()

    print(classification_report(y_test, y_pred_test))

____
____
____

## 1) Interpretabilidade de modelos

Nesta altura, já entendemos bem **o que** é um modelo, e como podemos construir modelos dos mais variados tipos.

Muitas vezes, no entanto, é de interesse que os modelos criados sejam **interpretáveis**, isto é, que seja possível **analisarmos por que o target $\hat{y}$ foi produzido pelo modelo**.

A necessidade ou não de interpretabilidade de modelos depende, muitas vezes, do problema de negócio específico. Pode haver certos problemas em que o objetivo é que tenhamos o modelo **com a melhor performance possível**, sem que haja necessidade de interpretarmos **o que** o modelos está fazendo. Se este for o caso, somos completamente livres para utilizar qualquer técnica que desejarmos, visando sempre aumentar a performance do modelo.

Em outros casos, no entanto, performance máxima não é o único objetivo: é necessário que os modelos produzidos também sejam interpretáveis, por diversos motivos, a citar alguns:

- Necessidade de extração de insights estratégicos a partir das estruturas aprendidas;
- Obrigação regulatória de interpretabilidade;
- Necessidade de adequação à regras de negócio particulares

Portanto, se interpretabiliade for uma questão importante, é importante que guiemos a construção de nossos modelos com este objetivo em mente. Para isso, há, essencialmente, duas abordagens possíveis:

> **Criar modelos naturalmente interpretáveis**: há modelos (que já conhecemos!), que são facilmente interpretáveis, devido à estrutura particular da hipótese. Assim, se interpretabilidade for algo importante, escolher estas hipóteses pode ser uma boa alternativa. 
<br><br>
No entanto, um ponto importantíssimo a ser considerado é que **há alguns procedimentos de pré-processamento** que podem obscurecer esta interpretabilidade natural (por exemplo: scalers, PCA, etc.). 
<br><br>
Portanto, se interpretabilidade de fato for uma questão, é muito importante que atenção seja tomada **até mesmo no pré-processamento** dos dados, mesmo que isso possa comprometer parte da performance do modelo;

> **Aplicar alguma técnica de explicabilidade de modelos**: se a busca por interpretabilidade acabar comprometendo muito a perfromance, é possível que sigamos com modelos que não sejam naturalmente interpretáveis, mas que possam ser interpretados por técnicas específicas que buscam interpretabilidade. Vamos estudar estas técnicas hoje!

Antes de mergulharmos neste assunto, um último comentário: a questão de interpretabilidade de modelos de ML é de extremo interesse pela comunidade científica, e cada vez mais tem ganhado espaço no mundo corporativo. De maneira mais geral, esta área é conhecida como [Explainable AI](https://en.wikipedia.org/wiki/Explainable_artificial_intelligence), e há grande esforço na direção de tornar AI uma área interpretável, o que leva a discussões bem fundamentais sobre estas tecnologias. Para quem se interessar, sugiro algumas leituras: [aqui, da IBM](https://www.ibm.com/watson/explainable-ai); [e aqui, do Google](https://cloud.google.com/explainable-ai).

E para quem realmente quiser mergulhar neste assunto, sugiro [este livro!](https://christophm.github.io/interpretable-ml-book/)

Agora, vamos detalhar um pouco mais as duas abordagens descritas acima!

____
____
____

## 2) Modelos naturalmente interpretáveis

Conforme discutimos acima, há modelos que são naturalmente explicáveis. Vamos discutir alguns deles:

____________

### Regressão Linear

O primeiro modelo que construímos no curso, foi um modelo de regressão linear para o preço de casas, com base em uma única variável:

> O nosso modelo final é dado por:
<br><br>
$$ y = f_H(x) =  1562.01 + 118.61\text{GrLiveArea}$$
<br><br>
Isto quer dizer que:
<br><br>
Aumentando a variável "GrLiveArea" em uma unidade faz com que o preço seja aumentado em USD 118.6!
<br><br>
O preço mínimo a ser pago, independente da área construída, é de 1562.01!

Mas mesmo modelos de regressão linear múltipla são interpretáveis! A hipótese é dada por:

$$ y = f_H(\vec{x}) = b_0 + \sum_{i=1}^n b_i X_i = b_0 + b_1 X_1 + b_2 X_2 + \cdots + b_n X_n $$

A interpretabilidade também é direta:

> Aumentando uma unidade da feature $X_i$, temos que o preço aumenta/diminui em $b_i$ unidades (a depender do sinal do coeficiente);
<br><br>
O preço fixo sempre é $b_0$.

Note, portanto, que simplesmente ao olharmos para os parâmetros da hipótese (`.coef_` e `.intercept_`), temos informações concretas e claríssimas sobre o modo como cada feature é utilizada para, conjuntamente, predizer o target. Esse é o mais simples exemplo de interpretabilidade!

**Observação**: também podemos olhar para o "tamanho" dos coeficientes para entender quais features são "mais importantes": coeficientes maiores (em valor absoluto) estão associados a "maior peso" na determinação do target. 

No entanto, a escala das features interfere nesta análise! Features com escala maior tendem a proporcionar coeficientes menores, e vice-versa. Por isso, se o objetivo é inspecionar importância de features na regressão linear, é importante que os dados sejam previamente escalados! [Este post](https://towardsdatascience.com/feature-importance-in-linear-models-four-often-neglected-but-crucial-pitfalls-e5c513e45b18) discute este ponto brevemente.

____________

### Árvores

Árvores são modelos naturalmente interpretáveis também: podemos inspecionar **o caminho que cada observação percorre do nó raiz até a folha** para entender exatamente qual foi o critério para a decisão, nó a nó!

Podemos inspecionar este caminho olhando para uma árvore plotada, ou então utilizando o método [decision_path!](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.decision_path) Vamos ver um exemplo rápido:

Para saber mais, como se aproveitar dessas estruturas, [clique aqui!](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html)

Adaptando o código da página acima, podemos construir uma função para interpretar a árvore:

____________

### KNN

Modelos KNN também são interpretáveis, em certa medida, pois podemos **olhar para os vizinhos que foram levados em conta pra tomar a decisão**, e com isso, podemos comparar estes vizinhos com a observação de teste, e, assim, interpretar a decisão!

Fazemos isso com o método [kneighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.kneighbors). Vamos ver um exemplo:

In [28]:
from sklearn.neighbors import KNeighborsClassifier

Vamos olhar para os vizinhos que foram utilizados na classificação:

Vamos olhar, por exemplo, apenas para a primeira observação:

Podemos usar os índices para inspecionar quais foram as observações utilizadas na classificação:

Faz sentido, pois:

Vamos criar uma função:

_____

O que fazemos no caso em que não é praticável utilizarmos alguns dos modelos mais simples e interpretáveis acima?

Nestes casos, o que vamos querer fazer é **treinar um modelo simples $g$** que seja capaz de **explicar um modelo complexo $f$**. Diremos então que $g$ é o **modelo explicativo** de $f$!

Veremos como fazer isso agora!

____
____
____

## 3) LIME

O LIME (**L**ocal **I**nterpretable **M**odel-agnostic **E**xplanations) é uma técnica que nos permite gerar modelos explicativos $g$ que são treinados **localmente** com o objetivo de explicar um modelo complexo $f$!

Considere o classificador à esquerda, que é o modelo original ($f$). Note a fronteira de decisão não-linear, que é um indicativo de dificuldade de explicabilidade. Caso queiramos explicar este modelo globalmente, teremos dificuldades, justo? (De fato, fronteiras de decisão não-lineares são dificilmente explicáveis!)

Por outro lado, **localmente**, em regiões próximas à fronteira de decisão, é possível **aproximarmos $f$ pelo modelo simples $g$, que é linear**. 

Com isso, **localmente**, fica fácil de explicar a decisão, com o modelo representado à direita!

<img src=https://deepandshallowml.files.wordpress.com/2019/11/lime_intuition_final.png width=600>

Os modelos lineares locais treinados pelo LIME também são conhecidos como surrogate models, e seu treinamento funciona da seguinte maneira:

- Um novo dataset de observações artificiais é criado (dados permutados), com base na distribuição das features na redondeza da observação a ser explicada;

- A distância entre estas observações e as observações reais são calculadas;

- O modelo é utilizado para predizer o `predict_proba` para estas novas observações;

- Escolhe-se as `m` features mais importantes, de acordo com os dados permutados;

- Um modelo linear é treinado com as `m` features, ponderando a similaridade entre samples e a observação a ser explicada.

O artigo original do LIME está disponível [aqui](https://arxiv.org/pdf/1602.04938.pdf) -- e ele é (talvez surpreendentemente) simples de ler!

E [neste post](https://towardsdatascience.com/decrypting-your-machine-learning-model-using-lime-5adc035109b5) há mais detalhes sobre o LIME e seu funcionamento!

Aqui, vamos ver, na prática, como aplicar o LIME!

Pra isso, existe a biblioteca [LIME](https://github.com/marcotcr/lime). Para instala-la:

`!pip install lime`

Vamos vê-la em ação!

Vamos construir um modelo bem mais complexo, para que faça sentido aplicarmos técnicas de explicabilidade!

Juntando tudo:

Acima, visualizamos o resultado do explainer!

O plot no centro é a principal informação para a interpretabilidade: ele exibe o valor dos coeficientes do modelo linear treinado localmente.

Importante: para o treinamento do modelo linear, as features numéricas são discretizadas em bins, cujos intervalos são indicados no plot.

As features são exibidas em ordem de importância, e é assim que somos capazes de descrever quais foram as features mais importantes na tomada de decisão, observação a observação!

Mudando a observação a ser explicada: a interpretação muda!

____
____
____

## 4) SHAP

Por fim, vamos dar uma olhada no SHAP (**SH**apley **A**dditive ex**P**lanation), um outro método muito utilizado para interpretabilidade.

o SHAP é um método baseado em um conceito de [teoria dos jogos](https://pt.wikipedia.org/wiki/Teoria_dos_jogos) conhecido como [Shapley values](https://christophm.github.io/interpretable-ml-book/shapley.html).

A ideia geral do método é encontrar **a importância das features para a predição** de modo bem explícito: para encontrar a importância de uma feature $x_i$, temos que:

- Treinar o modelo $f$ com todos os **subconjuntos possíveis** de features, **incluindo $x_i$**;
- Depois treinar o modelo $f$ com os mesmos subconjuntos, mas **excluindo $x_i$**.
- Depois, medimos a diferença entre os outputs de cada par de modelos.

Com a diferença entre os outputs, nós conseguimos medir **o impacto** da remoção daquela feature no output. Tomando uma espécie de **média** deste impacto dentre todos os subconjuntos, conseguimos ter a importância geral de $x_i$!

Obs.: o operacional do método é similar ao RFE, com a diferença de que aqui consideramos **todos** os subconjuntos com e sem a feature $x_i$!

Exemplo de subconjuntos:

Considere que temos as features $\vec{x} = (x_1, x_2, x_3, x_4)$, e que queremos estimar o impacto da feature $x_1$. 

Os subconjuntos possíveis **que incluem** $x_1$ são:

$ \{x_1\}$

$ \{x_1, x_2\}$

$ \{x_1, x_3\}$

$ \{x_1, x_4\}$

$ \{x_1, x_2, x_3\}$

$ \{x_1, x_2, x_4\}$

$ \{x_1, x_3, x_4\}$

$ \{x_1, x_2, x_3, x_4\}$

Os subconjuntos possíveis **que não incluem** $x_1$ são:

$ \{ \}$

$ \{x_2\}$

$ \{x_3\}$

$ \{x_4\}$

$ \{x_2, x_3\}$

$ \{x_2, x_4\}$

$ \{x_3, x_4\}$

$ \{x_2, x_3, x_4\}$

Note, portanto, que treinamos **16 modelos diferentes** para avaliar a importância de $x_1$

> No geral, para $n$ features, temos $2 \times 2^{n-1} = 2^n$ modelos diferentes que devem ser treinados para inspecionar a importância de cada feature, ou seja, $n \times 2^n$ modelos no total!

Portanto, fica claro que a utilização de Shapley values para a interpretação de modelos é algo **computacionalmente extremamente custoso**. 

É pensando nisso que o método SHAP vem à nossa salvação! :D

O artigo original do SHAP está [aqui](https://arxiv.org/pdf/1705.07874.pdf) -- esse é um pouquinho mais difícil, mas também é relativamente acessível!

E [neste post](https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30) há detalhes interessantes sobre o funcionamento do SHAP!

> Interlúdio matemático (conceitual): conforme dissemos, o SHAP é um método baseado em conceitos de teoria de jogos. Nesta teoria, há dois agentes muito importantes: **o jogo** e **os jogadores**. No contexto de interpretabilidade de ML, temos:
<br>
- Os outputs do modelo como o jogo;
<br><br>
- Os jogadores como as features a serem incluídas no modelo.
<br><br>
>A interação entre estes agentes é quantificada justamente pelo valor de Shapley. Vide os links acima para detalhes! 

Agora vamos ver, na prática, como aplicar o SHAP!

Pra isso, existe a biblioteca [SHAP](https://shap.readthedocs.io/en/latest/index.html). Para instala-la:

`!pip install shap`

Vamos vê-la em ação!

### Como interpretamos o gráfico de resumo do SHAP?

- No eixo y temos as features, ordenadas por importância;

- No eixo x temos os SHAP values -- valores positivos estão positivamente associados com o target 1, e vice-versa;

- Cada ponto é uma observação do dataset original;

- As cores estão associadas com os tamanhos (valor absoluto) da respectiva feature pra cada observação;

____
____
____